In [ ]:
#import libraries
import os
import math
import easydict
import itertools
import random
import csv

verbose = False

In [ ]:
def pair_file_generator(args):
    dataset_path = os.path.join(args.root,args.dataset_name)
    folds = args.folds
    match_list = []
    mismatch_list = []
    computation_check = 0
    if args.dataset_modality == 'folder':
        list_folders = [ind for ind in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path,ind))]
        for n,individual in enumerate(list_folders):
            if n//100 and (n//100) > computation_check: 
                computation_check = n//100
                print("Computing {}*100 individuals".format(computation_check))
            images = os.listdir(os.path.join(dataset_path,individual))
            N = len(images)
            if N < 2: 
                if verbose: print("Not enough images for {} in match".format(individual))
            else:
                C = int(math.factorial(N)/(math.factorial(2)*math.factorial(N-2)))
                if verbose: print("Combinations : {}".format(C))
                random.shuffle(images)
                list_combinations = list(itertools.combinations(images,2))
                index_random = random.sample(range(0,C),C)
                
                for index in index_random:
                    pair = list_combinations[index]
                    line = [individual,pair[0],pair[1]]
                    if verbose: print("Match {}".format(line))
                    match_list.append(line)
                    
                list_others = []
                list_mism_individual = []
                for m,mism_individual in enumerate(list_folders):
                    if not n == m:
                        list_others.append(os.listdir(os.path.join(dataset_path,mism_individual)))
                        list_mism_individual.append(
                            list(itertools.repeat(mism_individual,len(os.listdir(os.path.join(dataset_path,mism_individual))))))
                a_flat = [(i,j, item) for i,sublst in enumerate(list_others) for j, item in enumerate(sublst)]
                list_random = random.sample(a_flat, N)
                c = list(itertools.product(images, list_random))
                list_random = random.sample(c,C)
                for pair in list_random:
                    x,y,pair1 = pair[1]
                    line = [individual,pair[0],list_mism_individual[x][y],pair1]
                    if verbose: print("MisMatch {}".format(line))
                    mismatch_list.append(line)
    if args.dataset_modality == 'file':
        known_ind = []
        with open(os.path.join(dataset_path,'annotations_czoo.txt'),"r") as f:          
            x_split= [x.split(' ') for x in f.readlines()]
        all_images = [x[1] for x in x_split]
        ind = [x[3] for x in x_split]   

        for n,individual in enumerate(ind):
            if n//100 and (n//100) > computation_check: 
                computation_check = n//100
                print("Computing {}*100 individuals".format(computation_check))
            if not individual in known_ind:
                images_index = [nn for nn,n in enumerate(ind) if n == individual]
                images = [all_images[i] for i in images_index]
                N = len(images)
                if N < 2: 
                    if verbose: print("Not enough images for {} in match".format(individual))
                else:
                    C = int(math.factorial(N)/(math.factorial(2)*math.factorial(N-2)))
                    if verbose: print("Combinations : {}".format(C))
                    random.shuffle(images)                                                   
                    list_combinations = list(itertools.combinations(images,2))
                    index_random = random.sample(range(0,C),C)
                
                    for index in index_random:
                        pair = list_combinations[index]
                        line = [individual,pair[0],pair[1]]
                        if verbose: print("Match {}".format(line))
                        match_list.append(line)                      
                               
                    images_index = [nn for nn,n in enumerate(ind) if not n == individual]
                    list_others = [all_images[i] for i in images_index]
                    list_mism_individual = [ind[i] for i in images_index]
                    a_flat = [(i,item) for i, item in enumerate(list_others)]
                    list_random = random.sample(a_flat, N)
                    c = list(itertools.product(images, list_random))
                    list_random = random.sample(c,C)
                    for pair in list_random:
                        x,pair1 = pair[1]
                        line = [individual,pair[0],list_mism_individual[x],pair1]
                        if verbose: print("MisMatch {}".format(line))
                        mismatch_list.append(line)
                known_ind.append(individual)
    print(len(match_list))
    print(len(mismatch_list))
    K = len(match_list)

    pair_file_path = os.path.join(dataset_path,args.file_name)
    if not K//args.folds > (args.images_x_fold-1):
        images_x_fold = int(K//args.folds)
    else:
        images_x_fold = args.images_x_fold
    random.shuffle(match_list)
    random.shuffle(mismatch_list)
    for i in range(args.folds):
        print("Fold {}".format(i))
        initial_index = images_x_fold * i
        print("initial position {} final position {}".format(initial_index,initial_index+images_x_fold))
        with open(pair_file_path,"w") as f:
            writer = csv.writer(f,delimiter=',')   
            for match_line in itertools.islice(match_list,initial_index,initial_index+images_x_fold):
                if verbose: print(match_line)
                writer.writerows([match_line])
            for mismatch_line in itertools.islice(mismatch_list,initial_index,initial_index+images_x_fold):
                if verbose: print(mismatch_line)   
                writer.writerows([mismatch_line])
            

In [ ]:
args = easydict.EasyDict({
    "root":"/home/rita/JupyterProjects/EYE-SEA/DataSets/Verification",
    "file_name":"pairs.txt",
    "dataset_name":"lfw", #"chimpanzee_faces-master/datasets_cropped_chimpanzee_faces/data_CZoo",
    "annotation_file":"annotations_czoo.txt",
    "dataset_modality":"folder",
    "folds":10,
    "images_x_fold":300,
    "gpus":0       
})

pair_file_generator(args)